In [1]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore, auth
from firebase_admin.firestore import FieldFilter
from google.cloud.firestore_v1.document import DocumentReference
from datetime import datetime
import json 
from dotenv import load_dotenv
from random import randint
import os
import re
class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

load_dotenv()

cred = credentials.Certificate(os.getenv("firebase_secret"))
app = firebase_admin.initialize_app(cred)


db = firestore.client()


def load_json_file(filename):
    with open(filename) as fp:
        content = json.load(fp)
    return content


In [2]:
def process_reference(reference: DocumentReference):
    return reference.path

def process_array(array: list):
    return [process_reference(item) if type(item) == DocumentReference else item for item in array]

def process_dict(input_dict):
    output_dict = {}
    for key, value in input_dict.items():
        if type(value) == DocumentReference:
            output_dict[key] = process_reference(value)
        elif isinstance(value, dict):
            output_dict[key] = process_dict(value)
        elif isinstance(value, list):
            output_dict[key] = process_array(value)
        elif isinstance(value, datetime):
            output_dict[key] = value.isoformat()
        else:
            output_dict[key] = value
    return output_dict

# Itens

In [3]:
itens_stream = db.collection("items").stream()

itens = {} 

for item in itens_stream:
    itens[item.id] = process_dict(item.to_dict())

In [4]:
for key, val in itens.items():
    itens[key] = process_dict(val)


```python
import json
with open("json1123/itens.json") as f:
    itens = json.load(f)
armarios_altos = filter(lambda x: x["short_descricao"] == "ARMÁRIO ALTO FECHADO", itens.values())
armarios_altos = list(armarios_altos)
for armario in armarios_altos: 
    key = armario["key"]
    itens[str(key)]["detalhes"]["medidas"] = '160 x 50 x 80 (AxCxL)'
```

In [5]:
with open("json1123/itens.json", "w") as fp:
    json.dump(itens, fp, indent=2, ensure_ascii=False)

## Volumes

In [6]:
volumes_stream = db.collection("volumes").where(filter=FieldFilter("deleted", "==", False)).stream()

volumes = {}

for volume in volumes_stream:
    volumes[volume.id] = process_dict(volume.to_dict())


In [7]:
with open("json1123/volumes.json", "w") as fp:
    json.dump(volumes, fp, indent=2, ensure_ascii=False)

In [8]:
len(volumes)

5504

In [9]:
len(itens)

9033

In [10]:
errados = [x for x in volumes.values() if x["medidas"] == {"a": 1, "c": 2, "l":3} ]

In [11]:
len(errados)

162

In [12]:
Not, Yes = len([x for x in itens.values() if x["meta"]["volumado"] == False]), len([x for x in itens.values() if x["meta"]["volumado"] == True])

print(f"{bcolors.OKGREEN}Itens não volumados: {Not}{bcolors.ENDC}")
print(f"{bcolors.OKGREEN}Itens volumados: {Yes}{bcolors.ENDC}")
porcentagem = (Yes/(Not+Yes))*100
print(f"{bcolors.OKGREEN}Porcentagem: {porcentagem:.2f}%{bcolors.ENDC}")

Itens não volumados: 676
Itens volumados: 8357
Porcentagem: 92.52%


In [13]:
import pandas as pd

In [14]:
volumes = pd.DataFrame(volumes).T

In [15]:
volumes["Predio"] = volumes.origem.apply(lambda x: x.split("/")[1][0:2])

In [16]:
def validate_medidas(medidas: dict) -> bool:
    return isinstance(medidas["a"], int) and isinstance(medidas["c"], int) and isinstance(medidas["l"], int)

def parse_medidas(medidas: dict) -> dict:
    return {"a": int(medidas["a"]), "c": int(medidas["c"]), "l": int(medidas["l"])}

volumes["medidas"] = volumes.medidas.apply(parse_medidas)

In [17]:
volumes["cubagem"] = volumes.medidas.apply(lambda x: x["a"]*x["c"]*x["l"]/1000_000)

In [18]:
cubagem = pd.pivot_table(volumes, index=["Predio", "categoria"], values=["cubagem"], aggfunc=["sum","count"])



In [19]:
cubagem.to_excel("cubagem.xlsx", merge_cells=False)

In [20]:
volumes["Edificio"] = volumes.origem.apply(lambda x: x.split("/")[1][0])

In [21]:
cubagem = pd.pivot_table(volumes, index=["Edificio", "categoria"], values=["cubagem"], aggfunc=["sum","count"])


In [22]:
cubagem.to_excel("cubagem2.xlsx", merge_cells=False)

In [23]:
import re

In [24]:
ares = [x for x in itens.values() if re.match(".*CONDICIONAD.*", x["short_descricao"])]

In [25]:
def extract_btus(descricao: str):
    match = re.search(r".*?(\d*\.{0,1}\d+)\s*BTU.*", descricao)
    if match:
        return match.group(1).replace(".", "")
    else:
        return None

In [26]:
set([x["short_descricao"] for x in ares])

{'APARELHO DE AR CONDICIONADO',
 'CONDICIONADOR DE AR',
 'CONDICIONADOR DE AR 18000BTUS',
 'CONDICIONADOR DE AR 9000BTUS',
 'CONDICIONADOR DE AR SPLIT INVERTER',
 "CONDICIONADOR DE AR, TIPO SPLIT, 60.000 BTU's",
 'UNIDADE CONDENSADORA DE AR SPLIT INVERTER - CONDICIONADOR DE AR SPLIT, 12.000 BTUS, MARCA ELECTROLUX, MODELO BI12F'}

In [27]:
btus = list(map(lambda x: extract_btus(x["detalhes"]["descricao"]), ares))

In [28]:
predio1 = [x["origem"][0] for x in ares]

In [29]:
predio2 = [x["origem"][0:2] for x in ares]


In [30]:
codigo = [x["key"] for x in ares]

In [31]:
ares_df = pd.DataFrame({"codigo": codigo, "predio": predio1, "predio2": predio2, "btus": btus})

In [32]:
pd.pivot_table(ares_df, index=["predio", "btus"], values=["codigo"], aggfunc=["count"]).to_excel("ares.xlsx", merge_cells=False)